In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

from DataLoader import DataLoader, Batch
from Model import Model
import OutputProcessor

import cvutils

## Specify input-data directory, and load models 

In [ ]:
loader = DataLoader('../data/', Model.batchSize, Model.imgSize)

model = Model('../model/best_models/', mustRestore=True)

In [ ]:
## Preprocessing Step 1: --> Correct slant
for img_name in os.listdir('final_test_set/'):
    img = cv2.imread('final_test_set/'+img_name, cv2.IMREAD_GRAYSCALE)
    img = cvutils.correct_slant(img)
    cv2.imwrite('slant_corrected_set/'+img_name, img)

In [ ]:
# Get all the inference/test images in a batch (Refer 'Batch' class in 'DataLoader')
infer_batch = loader.getInferBatch()

# Get the predictions (targetSplits --> x-cordinates of vertical lines segmenting words into characters)
predictedSplits = model.inferBatch(infer_batch)
infer_batch.targetSplits = predictedSplits

# Apply Non Max Suppression to eliminate overlapping/low-confidence output Splits
# Rescale them to original image size
final_splits = OutputProcessor.apply_nms_and_rescale_outputs(infer_batch)

In [ ]:
# Iterate through every image in test/infer Samples and draw the corresponding splits
for i in range(len(loader.inferSamples)):
    img = cv2.imread('../data/test_words/'+loader.inferSamples[i])

    # img = cvutils.viz_splits_all(cv2.imread('../data/test_words/'+loader.inferSamples[4]), infer_batch.targetSplits[4], confidence_threshold=0.1)

    img = cvutils.viz_splits_final(img,final_splits[i])
    cv2.imwrite('final_splits/'+loader.inferSamples[i], img)